# Supervised fine tuning Gemma on chat Task
Recall that creating a ChatGPT at home involves 3 steps:

1. pre-training a large language model (LLM) to predict the next token on internet-scale data, on clusters of thousands of GPUs. One calls the result a "base model"
2. supervised fine-tuning (SFT) to turn the base model into a useful assistant
3. human preference fine-tuning which increases the assistant's friendliness, helpfulness and safety.

in this Notebook I will do the second step. This involves supervised fine-tuning (SFT for short), also called instruction tuning.

Supervised fine-tuning takes in a "base model" from step 1, i.e. a model that has been pre-trained on predicting the next token on internet text, and turns it into a "chatbot"/"assistant". This is done by fine-tuning the model on human instruction data

## Set-up environment

* BitsandBytes and PEFT for fine-tuning the model on consumer hardware, leveraging
* TRL, a library which includes useful Trainer classes for LLM fine-tuning.

In [ ]:
!pip install -q bitsandbytes trl peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
ERROR: pip's

We also install Flash

Attention, which speeds up the attention computations of the model.

In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.1-cp310-cp310-linux_x86_64.whl size=198447665 sha256=808523ff263d2fc1d3801147ad90d89dbfca67623a110c26592aef2064ea7b65
  Stored in directory: /root/.cache/pip/wheels/91/6a/38/f0faa036b4ac73a73247386f1ab1bb4cb4f6e72e6861a779f1
Successfully built flash-attn


## Load dataset


In [ ]:
from datasets import load_dataset

# based on config
raw_datasets = load_dataset("HuggingFaceH4/ultrachat_200k")

In [ ]:
raw_datasets

DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 207865
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 23110
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 256032
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 28304
    })
})

In [ ]:
raw_datasets['test_gen'][0]

{'prompt': 'This story begins with an end. In March 1991, after a three-month stay at Dr Bharat Vatwani’s Shraddha Nursing Home in Mumbai, Gangadhar Vinode, then 17, returned to his village near Wakad, in Pune. Earlier that year, the schizophrenic was found lying by a drain in Mumbai and had been picked up by staffers of Missionaries of Charity.\nWhen Dr Vatwani, who used to help the Missionaries of Charity with their work, saw him at the centre, he decided to take him under his wing. “He was not in good shape; I told them I wanted to try and cure him,” Dr Vatwani says. Towards the end of his stay at the nursing home, Vinode provided Dr Vatwani his address in Wakad, and the doctor rode down to Pune to inform his parents about their son. The next day, Vinode’s parents and grandfather arrived at the nursing home with tons of sweets, performed an aarti for Dr Vatwani, and left with their son.\nVinode, who has a shy smile and a childlike excitement, has just completed his third housing pro

In [ ]:
raw_datasets['test_sft'][1]

{'prompt': 'Rice tolerance to suboptimal low temperatures relies on the maintenance of the photosynthetic capacity.\nGazquez, A., Vilas, J. M., Colman Lerner, J. E., Maiale, S. J., Calzadilla, P. I., Menendez, A. B. and Rodriguez, A. A.\nLaboratorio de Fisiologia de Estres Abiotico en Plantas, Unidad de Biotecnologia 1, IIB-INTECH, CONICET, UNSAM, Chascomus, Argentina.\nCentro de Investigaciones y Desarrollo en Ciencias Aplicadas, FCEx, UNLP, Argentina.\nDepartamento de Biodiversidad y Biologia Experimental, FCEyN - UBA, INMIBO-CONICET, Buenos Aires, Argentina.\nLaboratorio de Fisiologia de Estres Abiotico en Plantas, Unidad de Biotecnologia 1, IIB-INTECH, CONICET, UNSAM, Chascomus, Argentina. Electronic address: andresrodriguez@conicet.gov.ar.\nThe purpose of this research was to identify differences between two contrasting rice cultivars in their response to suboptimal low temperatures stress. A transcriptomic analysis of the seedlings was performed and results were complemented with

The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do supervised fine-tuning (SFT), only the "train_sft" and "test_sft" splits are relevant for us.

In [ ]:
from datasets import DatasetDict


indices = range(0,100) # debugging mode

dataset_dict = {"train": raw_datasets["train_sft"].select(indices),
                "test": raw_datasets["test_sft"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 100
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 100
    })
})

Let's check one example. The important thing is that each example should contain a list of messages:

In [ ]:
example = raw_datasets["train"][0]
print(example.keys())

dict_keys(['prompt', 'prompt_id', 'messages'])


Each message is a dictionary containing 2 keys, namely:

* "role": specifies who the creator of the message is (could be "system", "assistant" or "user" - the latter referring to a human).
* "content": the actual content of the message.

Let's print out the sequence of messages for this training example:

In [ ]:
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))
  print('#'*10)

user                :  These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?
##########
assistant           :  This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.
##########
user                :  Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?
##########
assistant           :  Sure, here are t

## Load tokenizer
Next, we instantiate the tokenizer, which is required to prepare the text for the model. The model doesn't directly take strings as input, but rather `input_ids`, which represent integer indices in the vocabulary of a Transformer model.

- the padding token ID.
during fine-tuning, we will need to pad the (instruction, completion) pairs in order to create batches of equal length.

- the model max length: this is required in order to truncate sequences which are too long for the model. Here we decide to train on at most 2048 tokens.

the chat template. A [chat template](https://huggingface.co/blog/chat-templates) determines how each list of messages is turned into a tokenizable string, by adding special strings in between such as `<|user|>` to indicate a user message and `<|assistant|>` to indicate the chatbot's response. Here we define the default chat template, used by most chat models. See also the [docs](https://huggingface.co/docs/transformers/main/en/chat_templating).

In [ ]:
from transformers import AutoTokenizer

model_id='google/gemma-2b' #Could not tune it in 16 GB ram
tokenizer = AutoTokenizer.from_pretrained(model_id,token=token,trust_remote_code=True)
tokenizer.model_max_length

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/tokenizer_config.json


1000000000000000019884624838656

In [ ]:

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

## Apply chat template


In [ ]:
import re
import random
from multiprocessing import cpu_count

In [ ]:
def apply_chat_template(example,tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example


In [ ]:
column_names = list(raw_datasets["train"].features)


raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)


Applying chat template (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

Sample 81 of the processed training set:

<|system|>
<|endoftext|>
<|user|>
Write a JavaScript function that takes in one parameter and checks if that parameter is a perfect square. The function should return a boolean value - true if the parameter is a perfect square and false otherwise. You should test your function using a range of inputs and provide the output for each test case.<|endoftext|>
<|assistant|>
```
function isPerfectSquare(num) {
  if (num < 0) {
    return false;
  }
  else if (Math.sqrt(num) === Math.floor(Math.sqrt(num))) {
    return true;
  }
  else {
    return false;
  }
}

console.log(isPerfectSquare(16)); // true
console.log(isPerfectSquare(25)); // true
console.log(isPerfectSquare(144)); // true
console.log(isPerfectSquare(7)); // false
console.log(isPerfectSquare(-25)); // false
```<|endoftext|>
<|user|>
Great, the function looks good! Can you add some comments to explain what each line of code is doing?<|endoftext|>
<|assistant|>
Sure, here is the updated fu

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100
    })
})

## Define model arguments

Next, it's time to define the model arguments.

Here, some explanation is required regarding ways to fine-tune model.

### Full fine-tuning

Typically, one performs "full fine-tuning": this means that we will simply update all the weights of the base model during fine-tuning. This is then typically done either in full precision (float32), or mixed precision (a combination of float32 and float16). However, with ever larger models like LLMs, this becomes infeasible.


### LoRa, a PEFT method

Hence, some clever people at Microsoft have come up with a method called [LoRa](https://huggingface.co/docs/peft/conceptual_guides/lora) (low-rank adaptation). The idea here is that, rather than performing full fine-tuning, we are going to freeze the existing model and only add a few parameter weights to the model (called "adapters"), which we're going to train.

LoRa is what we call a parameter-efficient fine-tuning (PEFT) method. It's a popular method for fine-tuning models in a parameter-efficient way, by only training a few adapters, keeping the existing model untouched

### QLoRa, an even more efficient method

With regular LoRa, one would keep the base model in 32 or 16 bits in memory, and then train the parameter weights. However, there have been new methods developed to shrink the size of a model considerably, to 8 or 4 bits per parameter (we call this ["quantization"](https://huggingface.co/docs/transformers/main_classes/quantization)). Hence, if we apply LoRa to a quantized model (like a 4-bit model), then we call this QLoRa

In [ ]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments


In [ ]:
from transformers import BitsAndBytesConfig , AutoModelForCausalLM
import torch
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
device_map

{'': 0}

In [ ]:
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(token)

In [ ]:
#  for n, _ in model.named_parameters():
#     if not n.endswith("bias"):
#         print(n)


In [ ]:

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
)

model_kwargs = dict(
    #attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    #hidden_activation=True,
    quantization_config=quantization_config,
)

model = AutoModelForCausalLM.from_pretrained(model_id,**model_kwargs)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/config.json
Model config GemmaConfig {
  "_name_or_path": "google/gemma-2b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": false,
  "vocab_size": 256000
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapsho

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing GemmaForCausalLM.

All the weights of GemmaForCausalLM were initialized from the model checkpoint at google/gemma-2b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GemmaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2b/snapshots/68e273d91b1d6ea57c9e6024c4f887832f7b43fa/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 1,
  "pad_token_id": 0
}



In [ ]:
from json import JSONEncoder

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/gemma-2b-sft-lora'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    eval_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="gemma-2b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)


In [ ]:
trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argu

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3071 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


In [ ]:
train_result = trainer.train()

***** Running training *****
  Num examples = 61
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 128
  Total optimization steps = 1
  Number of trainable parameters = 14,745,600


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 

In [ ]:
trainer.push_to_hub()

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"""\ntrainable model parameters:{trainable_model_params}
                all model parameters: {all_model_params}
                percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"""


In [ ]:
print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(trainer.model)}\n')


PEFT model parameters to be updated:

trainable model parameters:14745600
                all model parameters: 1530013696
                percentage of trainable model parameters: 0.96%

